In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from nltk.corpus import wordnet
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction import text
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import wordnet as wn

c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# CSV 파일에서 데이터 불러오기
apps_data = pd.read_csv('cleaned_apps_data.csv')

C:\Users\user\AppData\Local\Temp\ipykernel_7596\3248333432.py:2: DtypeWarning: Columns (3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  apps_data = pd.read_csv('cleaned_apps_data.csv')


In [4]:
apps_data.columns

Index(['appId', 'title', 'url', 'score', 'price', 'free', 'category',
       'installs', 'description', 'icon'],
      dtype='object')

In [5]:
# 2. 앱 이름을 불용어 리스트로 처리
app_names = apps_data['title'].apply(lambda x: x.lower()).tolist()  # 앱 이름을 소문자로 변환하여 리스트로 수집


In [6]:
# 3. 추가적인 불용어 정의
additional_stopwords = ['pdf', 'file', 'app', 'lite', 'application']

In [7]:
# 4. 기본 불용어와 추가 불용어 결합
stop_words = list(text.ENGLISH_STOP_WORDS.union(app_names).union(additional_stopwords))


In [8]:
# 2. Sentence-BERT 모델 로드 및 문장 임베딩 생성
model = SentenceTransformer('all-MiniLM-L6-v2')  # Sentence-BERT 모델
embeddings = model.encode(apps_data['description'].values)  # 앱 설명에 대한 임베딩 생성


c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
# 3. K-Means 클러스터링 수행
n_clusters = 4  # 클러스터 수 설정
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
clusters = kmeans.fit_predict(embeddings)


c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [10]:
# 4. 클러스터 결과를 데이터프레임에 추가
apps_data['Cluster'] = clusters



In [11]:
# 7. TF-IDF 기반 주요 키워드 추출 함수
vectorizer = TfidfVectorizer(stop_words=stop_words, max_features=1000)
tfidf_matrix = vectorizer.fit_transform(apps_data['description'])

def get_top_keywords(tfidf_matrix, clusters, top_n=5):
    cluster_centers = np.zeros((np.unique(clusters).size, tfidf_matrix.shape[1]))
    
    for cluster in np.unique(clusters):
        cluster_centers[cluster] = tfidf_matrix[clusters == cluster].mean(axis=0)
    
    terms = vectorizer.get_feature_names_out()
    top_keywords = []
    
    for cluster in range(cluster_centers.shape[0]):
        center = cluster_centers[cluster]
        top_indices = center.argsort()[::-1][:top_n]
        keywords = [terms[i] for i in top_indices]
        top_keywords.append(keywords)
    
    return top_keywords

c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['0260', '084', '10', '100', '100km', '10bii', '119', '12306', '129', '12c', '13', '15', '15min', '17', '170m', '18', '1960', '1a', '1dm', '1money', '1st', '1tl', '1v1', '200', '2000', '2020', '2023', '2024', '2025', '21', '22', '23andme', '24', '24go', '250', '2d', '2e', '2fa', '2go', '2nd', '2ndline', '2number', '2redbeans', '2steps', '30', '300', '309', '33', '339', '350', '360', '365', '3commas', '3d', '3e', '3fun', '3rd', '3some', '3v3', '3way', '42', '4biddenknowledge', '4k', '4x', '50', '500', '504', '53', '5d', '5e', '5g', '5k', '60', '608', '60mph', '616', '75', '7now', '7shifts', '80', '80s', '83', '84', '85', '85c', '90', '91', '98', '991', '99acres', '_happy', 'a3d', 'aa', 'aaad', 'aaptiv', 'aarp', 'aawireless', 'aba', 'abc', 'abcmouse', 'abcs', 'abi

In [12]:
# 6. 클러스터별 상위 5개 키워드 추출
top_keywords_per_cluster = get_top_keywords(tfidf_matrix, clusters, top_n=5)



In [13]:
# 7. 키워드 기반으로 상위 카테고리 이름 생성
def generate_category_name(keywords):
    return " & ".join(keywords[:2])  # 상위 2개의 키워드를 연결하여 이름 생성



In [14]:
# 9. WordNet을 사용하여 키워드를 일반화하는 함수
from nltk.corpus import wordnet

In [15]:
def generalize_word(word):
    synsets = wordnet.synsets(word)
    if synsets:
        # 첫 번째 동의어 집합을 사용하여 일반화
        hypernyms = synsets[0].hypernyms()
        if hypernyms:
            return hypernyms[0].lemmas()[0].name()  # 가장 일반적인 상위 단어 반환
    return word  # 동의어 집합이 없으면 원래 단어 반환

In [16]:
# 10. 키워드 기반으로 상위 카테고리 이름 생성
def generate_category_name(keywords):
    generalized_keywords = [generalize_word(kw) for kw in keywords]
    return " & ".join(generalized_keywords[:2])  # 상위 2개의 일반화된 키워드를 연결

In [17]:
# 11. 클러스터별 상위 카테고리 이름 생성 및 출력
for cluster_num, keywords in enumerate(top_keywords_per_cluster):
    category_name = generate_category_name(keywords)
    print(f"Cluster {cluster_num} Name: {category_name}")


Cluster 0 Name: agency & representation
Cluster 1 Name: case & activity
Cluster 2 Name: linguistic_process & sacred_text
Cluster 3 Name: chemical_analysis & conversation


In [18]:
# 12. 클러스터 이름을 데이터프레임에 추가
for cluster_num, keywords in enumerate(top_keywords_per_cluster):
    category_name = generate_category_name(keywords)
    apps_data.loc[apps_data['Cluster'] == cluster_num, 'Super_Category'] = category_name


In [19]:
# 10. 결과 확인
print(apps_data[['title', 'category', 'description', 'Cluster', 'Super_Category']])

                                 title          category  \
0                                title          category   
1                      Google Messages     Communication   
2                               TikTok            Social   
3        Temu: Shop Like a Billionaire          Shopping   
4                              Threads            Social   
...                                ...               ...   
104737                 Apple tv Remote  Libraries & Demo   
104738               Ağlatan Hikayeler  Libraries & Demo   
104739  kubenav - Kubernetes Dashboard  Libraries & Demo   
104740                Why Not Compose!  Libraries & Demo   
104741            قوالب بوربوينت جاهزة  Libraries & Demo   

                                              description  Cluster  \
0                                             description        0   
1       Google Messages is the official Google app for...        3   
2       TikTok is THE destination for mobile videos. O...        0   

In [20]:
# clustering 'clustered_apps_data.csv' 파일로 저장
apps_data.to_csv('clustered_apps_data.csv', index=False)

In [21]:
# clustering 'clustered_apps_data.csv' 파일로 저장
apps_data.to_excel('clustered_apps_data.xlsx', index=False)

# WordNet

In [22]:
# 2. WordNet을 사용하여 키워드의 상위 개념을 찾는 함수
def get_hypernyms(word):
    synsets = wn.synsets(word)
    if synsets:
        hypernyms = synsets[0].hypernyms()
        if hypernyms:
            return hypernyms[0].lemmas()[0].name()  # 첫 번째 상위 개념 반환
    return word  # 상위 개념이 없으면 원래 단어 반환

In [23]:
# 3. 키워드를 상위 개념으로 변환하는 함수
def generalize_keywords(keywords):
    generalized_keywords = [get_hypernyms(word) for word in keywords]
    return generalized_keywords


In [24]:
# 4. Sentence-BERT 모델 로드 및 문장 임베딩 생성
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(apps_data['description'].values)


c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [25]:
# 5. K-Means 클러스터링 수행
n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
clusters = kmeans.fit_predict(embeddings)

c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [26]:
# 7. TF-IDF 기반 주요 키워드 추출
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
tfidf_matrix = vectorizer.fit_transform(apps_data['description'])

def get_top_keywords(tfidf_matrix, clusters, top_n=5):
    cluster_centers = np.zeros((np.unique(clusters).size, tfidf_matrix.shape[1]))
    
    for cluster in np.unique(clusters):
        cluster_centers[cluster] = tfidf_matrix[clusters == cluster].mean(axis=0)
    
    terms = vectorizer.get_feature_names_out()
    top_keywords = []
    
    for cluster in range(cluster_centers.shape[0]):
        center = cluster_centers[cluster]
        top_indices = center.argsort()[::-1][:top_n]
        keywords = [terms[i] for i in top_indices]
        top_keywords.append(keywords)
    
    return top_keywords

In [27]:
# 8. 클러스터별 상위 5개 키워드 추출
top_keywords_per_cluster = get_top_keywords(tfidf_matrix, clusters, top_n=5)

In [28]:
# 9. 상위 개념 기반 카테고리 이름 생성
def generate_category_name(keywords):
    generalized_keywords = generalize_keywords(keywords)
    return " & ".join(set(generalized_keywords[:2]))  # 중복되지 않은 상위 2개의 단어

In [29]:
# 10. 클러스터별 상위 카테고리 이름 생성 및 출력
for cluster_num, keywords in enumerate(top_keywords_per_cluster):
    category_name = generate_category_name(keywords)
    print(f"Cluster {cluster_num} Name: {category_name}")


Cluster 0 Name: electronic_equipment & app
Cluster 1 Name: chemical_analysis & conversation
Cluster 2 Name: linguistic_process & sacred_text
Cluster 3 Name: representation & agency
Cluster 4 Name: consequence & app


# Cosine Similarity

In [30]:
# 5. TF-IDF 기반 주요 키워드 추출
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
tfidf_matrix = vectorizer.fit_transform(apps_data['description'])

def get_top_keywords(tfidf_matrix, clusters, top_n=5):
    cluster_centers = np.zeros((np.unique(clusters).size, tfidf_matrix.shape[1]))
    
    for cluster in np.unique(clusters):
        cluster_centers[cluster] = tfidf_matrix[clusters == cluster].mean(axis=0)
    
    terms = vectorizer.get_feature_names_out()
    top_keywords = []
    
    for cluster in range(cluster_centers.shape[0]):
        center = cluster_centers[cluster]
        top_indices = center.argsort()[::-1][:top_n]
        keywords = [terms[i] for i in top_indices]
        top_keywords.append(keywords)
    
    return top_keywords

In [31]:
# 6. 클러스터별 상위 5개 키워드 추출
top_keywords_per_cluster = get_top_keywords(tfidf_matrix, clusters, top_n=5)

In [32]:
# 7. 각 키워드를 임베딩하여 벡터 생성
def get_keyword_embedding(keywords):
    return model.encode(keywords)

In [33]:
# 8. Cosine Similarity를 사용하여 중심 단어 선택
def select_representative_word(keywords):
    keyword_embeddings = get_keyword_embedding(keywords)
    similarity_matrix = cosine_similarity(keyword_embeddings)

    # 각 단어의 유사도 합 계산
    similarity_sums = similarity_matrix.sum(axis=1)
    
    # 유사도 합이 가장 큰 단어를 대표 단어로 선택
    representative_idx = np.argmax(similarity_sums)
    return keywords[representative_idx]

In [34]:
# 9. 클러스터별 대표 단어 생성
def generate_category_name(keywords):
    return select_representative_word(keywords)

In [35]:
# 10. 클러스터별 대표 단어 생성 및 출력
for cluster_num, keywords in enumerate(top_keywords_per_cluster):
    category_name = generate_category_name(keywords)
    print(f"Cluster {cluster_num} Name: {category_name}")


Cluster 0 Name: phone
Cluster 1 Name: chat
Cluster 2 Name: reading
Cluster 3 Name: design
Cluster 4 Name: card


In [36]:
# 11. 클러스터 이름을 데이터프레임에 추가
for cluster_num, keywords in enumerate(top_keywords_per_cluster):
    category_name = generate_category_name(keywords)
    apps_data.loc[apps_data['Cluster'] == cluster_num, 'Super_Category'] = category_name


In [37]:
# 결과 확인
print(apps_data[['title', 'category', 'description', 'Cluster', 'Super_Category']])


                                 title          category  \
0                                title          category   
1                      Google Messages     Communication   
2                               TikTok            Social   
3        Temu: Shop Like a Billionaire          Shopping   
4                              Threads            Social   
...                                ...               ...   
104737                 Apple tv Remote  Libraries & Demo   
104738               Ağlatan Hikayeler  Libraries & Demo   
104739  kubenav - Kubernetes Dashboard  Libraries & Demo   
104740                Why Not Compose!  Libraries & Demo   
104741            قوالب بوربوينت جاهزة  Libraries & Demo   

                                              description  Cluster  \
0                                             description        0   
1       Google Messages is the official Google app for...        3   
2       TikTok is THE destination for mobile videos. O...        0   

In [38]:
from sentence_transformers import SentenceTransformer

# Sentence-BERT 모델 로드 (여기서는 'all-MiniLM-L6-v2' 모델 사용)
model = SentenceTransformer('all-MiniLM-L6-v2')

# Cleaned_Description 열에서 전처리된 텍스트를 기반으로 임베딩 생성
embeddings = model.encode(apps_data['description'].values)

# 임베딩 확인
print(embeddings.shape)  # (데이터 개수, 임베딩 차원 수)
print(embeddings)  # 임베딩 결과 확인

c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


(104742, 384)
[[-0.01717039  0.14092764 -0.00787549 ...  0.11699603 -0.00353066
  -0.01411287]
 [-0.0468857  -0.03969275  0.12062863 ...  0.03706462 -0.04091983
  -0.04607415]
 [-0.08838934 -0.01335814  0.04051472 ...  0.06879997 -0.01539475
   0.04982326]
 ...
 [ 0.00708288  0.06026745 -0.00740675 ...  0.00765995 -0.01902615
   0.03554336]
 [-0.08424091 -0.09615403  0.02267672 ...  0.03911785  0.04537113
   0.01596979]
 [-0.01768885  0.04914334  0.02923925 ...  0.07117655 -0.03629033
  -0.0103277 ]]


In [39]:
from sklearn.cluster import KMeans

# KMeans 클러스터링
num_clusters = 5
kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(embeddings)

# 클러스터 할당 결과
clusters = kmeans.labels_
print(clusters)

c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


[2 1 0 ... 0 2 2]


## Sentence Bert 저장

In [40]:
import joblib
from sentence_transformers import SentenceTransformer

# Sentence-BERT 모델 로드
model = SentenceTransformer('all-MiniLM-L6-v2')

# 예시 텍스트 임베딩 (앱 설명)
app_descriptions = ["카카오톡은 무료 메시징 앱입니다.", "네이버는 한국의 대표 포털 사이트입니다."]
embeddings = model.encode(app_descriptions)

# 임베딩 저장
joblib.dump(embeddings, 'app_embeddings2.pkl')


c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


['app_embeddings2.pkl']

In [47]:
import joblib
from sklearn.cluster import KMeans

# K-Means 모델 학습 (n_clusters는 군집 개수)
kmeans = KMeans(n_clusters=2, random_state=42)

# kmeans.cluster_centers_=kmeans.cluster_centers_.astype(float)

kmeans.fit(embeddings)

# K-Means 모델 저장
joblib.dump(kmeans, 'kmeans_model_4.pkl')


c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


['kmeans_model_4.pkl']

In [48]:
print(embeddings.shape)

(2, 384)


In [ ]:
import sqlite3
import pandas as pd

# 데이터베이스 연결
conn = sqlite3.connect('app_data.db')

# 예시 앱 데이터프레임 생성
apps_data = pd.DataFrame({
    'AppID': ['com.kakao.talk', 'com.nhn.android.search', 'com.facebook.katana'],
    'Name': ['카카오톡', '네이버', '페이스북'],
    'Description': ['카카오톡은 무료 메시징 앱입니다.', '네이버는 한국의 대표 포털 사이트입니다.', '페이스북은 세계 최대의 소셜 네트워크입니다.']
})

# 데이터프레임을 데이터베이스에 저장
apps_data.to_sql('apps', conn, if_exists='replace', index=False)

# 데이터베이스 연결 종료
conn.close()
